In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq

from model import *

Select a particular dataset and model run, then create an instance of the Inpatients Model class and load the datset.

In [2]:
dataset = "data/synthetic"
model_run = f"test/20220110_104353"
m = InpatientsModel(f"{dataset}/results/{model_run}")
ip = pq.read_pandas(f"{dataset}/ip.parquet").to_pandas()
ip.drop(["admigrp"], axis = "columns", inplace = True)

We can now perform a model run. The `0` is the model run number. This alters the random seed used for the PRNG, meaning each time we run the model with this value we will get the same results.

In [3]:
selected_variant, results = m.run(0)
results

,rn,speldur,classpat,admission_avoidance_strategy,los_reduction_strategy
0,1,0.0,2,NULL,bads_daycase
1,1,0.0,2,NULL,bads_daycase
2,735,1.0,1,NULL,improved_discharge_planning_non-elective
3,735,1.0,1,NULL,improved_discharge_planning_non-elective
4,841,4.0,1,ambulatory_care_conditions_chronic,NULL
...,...,...,...,...,...
148798,127945,0.0,2,NULL,NULL
148799,128227,0.0,2,NULL,NULL
148800,128227,0.0,2,NULL,NULL
148801,128833,13.0,1,ambulatory_care_conditions_chronic,NULL


We can join the results back to the full dataset to get all columns.

In [4]:
merged_results = ip.drop(["classpat", "speldur"], axis = "columns").merge(results, on = "rn")

We can now compare the base data to the modelled results.

In [5]:
# mean length of stay in the baseline vs results
(np.mean(ip.speldur), np.mean(merged_results.speldur))

(2.1150341597326157, 1.7143538772739797)

In [6]:
# number of rows in the baseline vs results
(len(ip.index), len(merged_results.index))

(128953, 148803)

In [7]:
# count by patient classification.
# Note: -1 indicates that a row has moved from the inpatients dataset to outpatients
pd.merge(
  ip.value_counts("classpat").to_frame("baseline"),
  merged_results.value_counts("classpat").to_frame("results"),
  left_index = True,
  right_index = True,
  how = "outer"
).fillna(0)

,baseline,results
classpat,,
-1,0.0,323
1,69522.0,76188
2,47833.0,58117
3,10571.0,13073
4,8.0,7
5,1019.0,1095
